In [ ]:
from collections import abc

import numpy as np
import scipy
from matplotlib import pyplot as plt

ndarray = np.ndarray

In [ ]:
def get_block(k: int, omega: float, M: ndarray, C: ndarray, K: ndarray):
    return -k * omega**2 * M + 1j * k * omega * C + K


def get_A(NH: int, omega: float, M: ndarray, C: ndarray, K: ndarray):
    return scipy.sparse.block_diag(
        [get_block(k, omega, M, C, K) for k in range(0, NH + 1)]
    ).tocsr()


def get_b_ext(
    NH: int,
    N: int,
    ks: abc.Iterable[int],
    dofs: abc.Iterable[int],
    is_cosines: abc.Iterable[bool],
    coefficients: abc.Iterable[float],
):
    """Return the exponential Fourier coefficients of the external force given
    cosine and sine coefficients.

    If an index is specified more than once (e.g., for both a cosine and sine
    coefficient), all corresponding coefficients are applied via addition.

    Parameters
    ----------
    NH
        The number of assumed harmonics, i.e., 0, 1, ..., N_H
    N
        The number of degrees of freedom in the system
    ks
        The indices corresponding to each coefficient in `coefficients`
    ks
        The indices corresponding to each coefficient in `coefficients`
    is_cosines
        Whether each corresponding coefficient in `coefficients` is cosine
    coefficients
        The coefficients of cosine and/or sine in the external force
    """
    if any([k > NH for k in ks]):
        raise ValueError(
            "At least one specified harmonic index is greater than NH."
        )
    if any([dof >= N for dof in dofs]):
        raise ValueError(
            "At least one specified degree of freedom is greater than or equal"
            " to N."
        )

    b_ext = np.zeros(N * (NH + 1), dtype=complex)

    for k, dof, is_cosine, coefficient in zip(
        ks, dofs, is_cosines, coefficients
    ):
        if k == 0:
            b_ext[dof] += coefficient

        elif is_cosine:
            b_ext[N * k + dof] += coefficient / 2
        else:
            b_ext[N * k + dof] += -1j * coefficient / 2

    return b_ext

In [ ]:
NH = 3
N = 2
m1, m2 = 1, 2
c1, c2, c3 = 1, 0.5, 2
k1, k2, k3 = 1, 0.5, 2

M = np.diag([m1, m2])
C = np.array([[c1 + c2, -c2], [-c2, c2 + c3]])
K = np.array([[k1 + k2, -k2], [-k2, k2 + k3]])

omega = 2

In [ ]:
A = get_A(NH, omega, M, C, K)

b_ext = get_b_ext(NH, N, (1,), (0,), (1,), (2,))
b_ext

array([0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j])

In [ ]:
np.linalg.solve(A.toarray(), b_ext)

array([ 0.        +0.j        ,  0.        +0.j        ,
       -0.16356568-0.18605943j, -0.03360178+0.01610664j,
        0.        -0.j        ,  0.        -0.j        ,
        0.        -0.j        ,  0.        -0.j        ])